In [2]:
import numpy as np
#from autograd import grad
import matplotlib.pyplot as plt

#import tensorflow as tf
#import torch
np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

theta_list = []
checkpoint_list = []

year = [2014]
period = ['may-jun'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')

    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')

    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')

    N = X['N']

    dist = X['distance']
    tI1 = X['tI1'].reshape(N,1)
    tI2 = X['tI2'].reshape(N,1)
    sI2 = X['sI2'].reshape(N,1)
    
    y_apr = X['y_apr'].reshape(N,1)
    y_may = X['y_may'].reshape(N,1)
    y_jun = X['y_jun'].reshape(N,1)
    y_jul = X['y_jul'].reshape(N,1)

    n_apr = X['n_apr'].reshape(N,1)
    n_may = X['n_may'].reshape(N,1)
    n_jun = X['n_jun'].reshape(N,1)
    n_jul = X['n_jul'].reshape(N,1)

    a_apr = X['a_apr'].reshape(N,1)
    a_may = X['a_may'].reshape(N,1)
    a_jun = X['a_jun'].reshape(N,1)
    a_jul = X['a_jul'].reshape(N,1)

    w_apr = X['wind_apr']
    w_may = X['wind_may']
    w_jun = X['wind_jun']
    w_jul = X['wind_jul']

    sI1_apr = X['sI1_apr'].reshape(N,1)
    sI1_may = X['sI1_may'].reshape(N,1)
    sI1_jun = X['sI1_jun'].reshape(N,1)
    sI1_jul = X['sI1_jul'].reshape(N,1)

    s_apr = X['s_apr'].reshape(N,1)
    s_may = X['s_may'].reshape(N,1)
    s_jun = X['s_jun'].reshape(N,1)
    s_jul = X['s_jul'].reshape(N,1)

    for period in period:

        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun
        
        # Create a matrix with N identical rows
        a_lag_tile = np.tile(a_lag, (N, 1, 1))
        y_lag_tile = np.tile(y_lag, (N, 1, 1))
        n_lag_tile = np.tile(n_lag, (N, 1, 1))
        s_lag_tile = np.tile(s_lag, (N, 1, 1))
        w_lag_tile = np.tile(w_lag, (N, 1, 1))
        dist_tile = np.tile(dist, (N, 1, 1))
        sI1_lag_tile = np.tile(sI1_lag, (N, 1, 1))
        sI1_lag_tile = sI1_lag_tile.astype(float)
        
        sI2_tile = np.tile(sI2, (N, 1, 1))
        sI2_tile = sI2_tile.astype(float)
        
        # Subtract the diagonal elements (i.e., set them to zero)
        a_lag_tile[np.arange(N), np.arange(N), 0] = np.nan
        y_lag_tile[np.arange(N), np.arange(N), 0] = np.nan
        n_lag_tile[np.arange(N), np.arange(N), 0] = np.nan
        s_lag_tile[np.arange(N), np.arange(N), 0] = np.nan
        w_lag_tile[np.arange(N), np.arange(N), :] = np.nan
        dist_tile[np.arange(N), np.arange(N), :] = np.nan
        sI1_lag_tile[np.arange(N), np.arange(N), 0] = np.nan
        sI2_tile[np.arange(N), np.arange(N), 0] = np.nan

In [72]:
dist[:,0]

array([ 0.        ,  0.08173764,  0.04727476,  0.31602146,  0.1687085 ,
        0.36312134,  0.31127086,  4.21220746,  4.981124  ,  4.35931709,
        4.94170305,  4.71978656,  4.95114477,  4.52304184,  0.17334079,
        0.31499975,  0.47533891,  5.94210092,  6.10533966,  6.06603606,
        0.34761678,  0.35211129,  6.06819212,  5.96251658,  5.17547491,
        4.28303658,  4.40591875,  5.24737279,  6.10258065,  6.11389283,
        6.2505615 ,  5.73804878,  5.88270043, 16.48139883, 20.79361515,
       16.28884574, 15.8661312 , 16.20835686, 17.45834743, 16.11677297,
       16.46063489, 18.50078383, 18.76184115, 17.10243274, 16.65697048,
       16.9486449 , 16.50292262, 16.97609703, 16.72238316, 17.84086199,
       17.66100676, 16.51132605, 17.48514137, 10.08801582, 10.13533725,
       10.06708349, 10.03471643,  9.91777973,  9.72293684,  9.55786676,
        9.40992433,  9.13578275, 10.31213186, 10.00552511,  2.95237663,
        3.0117985 ,  2.86037331,  2.67753426,  1.78882951,  1.13

In [71]:
dist_tile[0][:,0]

array([        nan,  0.08173764,  0.04727476,  0.31602146,  0.1687085 ,
        0.36312134,  0.31127086,  4.21220746,  4.981124  ,  4.35931709,
        4.94170305,  4.71978656,  4.95114477,  4.52304184,  0.17334079,
        0.31499975,  0.47533891,  5.94210092,  6.10533966,  6.06603606,
        0.34761678,  0.35211129,  6.06819212,  5.96251658,  5.17547491,
        4.28303658,  4.40591875,  5.24737279,  6.10258065,  6.11389283,
        6.2505615 ,  5.73804878,  5.88270043, 16.48139883, 20.79361515,
       16.28884574, 15.8661312 , 16.20835686, 17.45834743, 16.11677297,
       16.46063489, 18.50078383, 18.76184115, 17.10243274, 16.65697048,
       16.9486449 , 16.50292262, 16.97609703, 16.72238316, 17.84086199,
       17.66100676, 16.51132605, 17.48514137, 10.08801582, 10.13533725,
       10.06708349, 10.03471643,  9.91777973,  9.72293684,  9.55786676,
        9.40992433,  9.13578275, 10.31213186, 10.00552511,  2.95237663,
        3.0117985 ,  2.86037331,  2.67753426,  1.78882951,  1.13

In [70]:
w_lag[:, 0]

array([0.        , 0.02248995, 0.00662248, 0.0326708 , 0.0218138 ,
       0.00938308, 0.03003079, 0.01667218, 0.01701338, 0.01689585,
       0.01862485, 0.01852546, 0.01873603, 0.01681696, 0.00678086,
       0.02194052, 0.02566796, 0.0224682 , 0.02164904, 0.01959695,
       0.02119097, 0.03095749, 0.01718702, 0.02063743, 0.01938343,
       0.01891767, 0.0191342 , 0.02087727, 0.01663278, 0.01026286,
       0.01645465, 0.01015198, 0.01649842, 0.00765829, 0.00690899,
       0.00762512, 0.00889952, 0.00725284, 0.00536449, 0.00467987,
       0.00748682, 0.00642059, 0.00658914, 0.01009279, 0.0114314 ,
       0.01122291, 0.01144868, 0.01033175, 0.01098708, 0.00901219,
       0.00894552, 0.01099142, 0.00901247, 0.00498172, 0.00494121,
       0.00945305, 0.00926055, 0.01047521, 0.01091334, 0.01056212,
       0.00587598, 0.00558788, 0.00856882, 0.00437667, 0.01978611,
       0.01980495, 0.01931544, 0.0137516 , 0.02248399, 0.0329257 ,
       0.02273099, 0.02492186, 0.02560626, 0.01736334, 0.02038

In [75]:
w_lag_tile[0][:,0]

array([       nan, 0.02248995, 0.00662248, 0.0326708 , 0.0218138 ,
       0.00938308, 0.03003079, 0.01667218, 0.01701338, 0.01689585,
       0.01862485, 0.01852546, 0.01873603, 0.01681696, 0.00678086,
       0.02194052, 0.02566796, 0.0224682 , 0.02164904, 0.01959695,
       0.02119097, 0.03095749, 0.01718702, 0.02063743, 0.01938343,
       0.01891767, 0.0191342 , 0.02087727, 0.01663278, 0.01026286,
       0.01645465, 0.01015198, 0.01649842, 0.00765829, 0.00690899,
       0.00762512, 0.00889952, 0.00725284, 0.00536449, 0.00467987,
       0.00748682, 0.00642059, 0.00658914, 0.01009279, 0.0114314 ,
       0.01122291, 0.01144868, 0.01033175, 0.01098708, 0.00901219,
       0.00894552, 0.01099142, 0.00901247, 0.00498172, 0.00494121,
       0.00945305, 0.00926055, 0.01047521, 0.01091334, 0.01056212,
       0.00587598, 0.00558788, 0.00856882, 0.00437667, 0.01978611,
       0.01980495, 0.01931544, 0.0137516 , 0.02248399, 0.0329257 ,
       0.02273099, 0.02492186, 0.02560626, 0.01736334, 0.02038

In [11]:
dist[0].shape

(104,)

In [174]:
arr1 = np.array([[1, 2, 3], 
                 [1, 2, 3], 
                 [1, 2, 3]])

arr2 = np.array([[3, 2, 1],
                 [3, 2, 1],
                 [3, 2, 1]])

# Tile the array to make it dimensions 3x3, with tiled_arr[0] = tiled_arr[1] = tiled_arr[2] = arr
tiled_arr1 = np.tile(arr, (3, 1, 1)).astype(float)
tiled_arr2 = np.tile(arr2, (3, 1, 1)).astype(float)

# Fill the diagonals of each 3x3 matrix with 0
tiled_arr1[np.arange(3), np.arange(3), :] = np.nan
tiled_arr2[np.arange(3), np.arange(3), :] = np.nan

print(arr1 * np.exp(arr2))
print(tiled_arr1 * np.exp(tiled_arr2))

[[20.08553692 14.7781122   8.15484549]
 [20.08553692 14.7781122   8.15484549]
 [20.08553692 14.7781122   8.15484549]]
[[[        nan         nan         nan]
  [20.08553692 14.7781122   8.15484549]
  [20.08553692 14.7781122   8.15484549]]

 [[20.08553692 14.7781122   8.15484549]
  [        nan         nan         nan]
  [20.08553692 14.7781122   8.15484549]]

 [[20.08553692 14.7781122   8.15484549]
  [20.08553692 14.7781122   8.15484549]
  [        nan         nan         nan]]]


## start test

In [118]:
yard = 3

mask = np.arange(N) != yard # mask out the current yard i

dispersal_component1 = 1 * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-1 * s_lag[mask]) * w_lag[:, yard][mask])

dispersal_component_fast = 1 * np.sum(np.nansum((a_lag_tile * (y_lag_tile / n_lag_tile)) * np.exp(-1 * s_lag_tile) * w_lag_tile[yard][:, yard], axis=1)[yard])

print(dispersal_component1)
print(dispersal_component_fast)

6.585308778550257
6.585308778550257


In [163]:
yard = 3

mask = np.arange(N) != yard # mask out the current yard i
                
dispersal_component1 = 1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-1 * s_lag[mask]) * (w_lag[:, yard][mask] * np.exp(-1 * dist[:, yard][mask])) * sI1_lag[mask]))
dispersal_component_sum = np.nansum((a_lag_tile * (y_lag_tile / n_lag_tile)) * np.exp(-1 * s_lag_tile) * (w_lag_tile[yard][:, yard] * np.exp(-1 * dist_tile[yard][:, yard])) * sI1_lag_tile, axis=1)
dispersal_component_fast = 1 * np.sum(dispersal_component_sum, axis=1)

print(dispersal_component1)
print(dispersal_component_fast)

0.46848868968011204
[0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.44014455 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46639677 0.46848869 0.46848869 0.46848869 0.31301276 0.41353719
 0.46848869 0.46848869 0.44042719 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.468488

In [168]:
yard = 3

mask = np.arange(N) != yard # mask out the current yard i
                
dispersal_component1 = 1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-1 * s_lag[mask]) * (w_lag[mask] * np.exp(-1 * dist[mask])) * sI1_lag[mask]), axis=1)
dispersal_component_sum = np.nansum((a_lag_tile * (y_lag_tile / n_lag_tile)) * np.exp(-1 * s_lag_tile) * (w_lag_tile[yard][:, yard] * np.exp(-1 * dist_tile[yard][:, yard])) * sI1_lag_tile, axis=1)
dispersal_component_fast = 1 * np.sum(dispersal_component_sum, axis=1)

print(dispersal_component1)
print(dispersal_component_fast)

0.22538133133581184
[0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.44014455 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46639677 0.46848869 0.46848869 0.46848869 0.31301276 0.41353719
 0.46848869 0.46848869 0.44042719 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869 0.46848869
 0.46848869 0.46848869 0.46848869 0.468488

In [80]:
w_lag[:, yard][mask].shape

(103,)

In [81]:
w_lag_tile[yard][:, yard].shape

(104,)

In [44]:
w_lag[yard][mask]

array([0.03003079, 0.03035362, 0.02942905, 0.01865541, 0.02973284,
       0.02163469, 0.03032501, 0.03024608, 0.03035869, 0.03036341,
       0.03036988, 0.03030359, 0.03036462, 0.02624629, 0.01925518,
       0.01939869, 0.02616986, 0.02676586, 0.02584159, 0.02620029,
       0.01950871, 0.02872241, 0.02898103, 0.02952158, 0.03006084,
       0.02961793, 0.02817557, 0.02995509, 0.03020259, 0.02962002,
       0.02987373, 0.029486  , 0.02631021, 0.02614781, 0.02610729,
       0.02654172, 0.02643493, 0.02533306, 0.02627535, 0.02599544,
       0.02848623, 0.02861132, 0.03024026, 0.03029461, 0.03024669,
       0.0302981 , 0.03016655, 0.03018085, 0.03014628, 0.0301219 ,
       0.03018213, 0.03014612, 0.02604456, 0.02598738, 0.02607955,
       0.02622026, 0.02645918, 0.02674474, 0.02702253, 0.02724888,
       0.02689886, 0.02502917, 0.02512771, 0.00659713, 0.00640316,
       0.0068268 , 0.00717625, 0.00356306, 0.00810428, 0.00454989,
       0.00396818, 0.0040708 , 0.00525576, 0.00358965, 0.00359

In [57]:
w_lag[yard]

array([0.03003079, 0.03035362, 0.02942905, 0.01865541, 0.02973284,
       0.02163469, 0.        , 0.03032501, 0.03024608, 0.03035869,
       0.03036341, 0.03036988, 0.03030359, 0.03036462, 0.02624629,
       0.01925518, 0.01939869, 0.02616986, 0.02676586, 0.02584159,
       0.02620029, 0.01950871, 0.02872241, 0.02898103, 0.02952158,
       0.03006084, 0.02961793, 0.02817557, 0.02995509, 0.03020259,
       0.02962002, 0.02987373, 0.029486  , 0.02631021, 0.02614781,
       0.02610729, 0.02654172, 0.02643493, 0.02533306, 0.02627535,
       0.02599544, 0.02848623, 0.02861132, 0.03024026, 0.03029461,
       0.03024669, 0.0302981 , 0.03016655, 0.03018085, 0.03014628,
       0.0301219 , 0.03018213, 0.03014612, 0.02604456, 0.02598738,
       0.02607955, 0.02622026, 0.02645918, 0.02674474, 0.02702253,
       0.02724888, 0.02689886, 0.02502917, 0.02512771, 0.00659713,
       0.00640316, 0.0068268 , 0.00717625, 0.00356306, 0.00810428,
       0.00454989, 0.00396818, 0.0040708 , 0.00525576, 0.00358

In [58]:
w_lag_tile[yard][:,yard]

array([0.01496664, 0.01803828, 0.01319707, 0.00736473, 0.02037705,
       0.00436191,        nan, 0.01649746, 0.01685304, 0.0167337 ,
       0.01847333, 0.01837138, 0.01857027, 0.01666383, 0.00851785,
       0.02252844, 0.0090069 , 0.02241838, 0.02157486, 0.01951371,
       0.02307274, 0.00927143, 0.01697919, 0.02045788, 0.01919303,
       0.01871102, 0.01894209, 0.02071475, 0.0164964 , 0.01017503,
       0.01631591, 0.01003268, 0.01631152, 0.00777179, 0.00700223,
       0.00774873, 0.00896119, 0.00736114, 0.00546825, 0.00475364,
       0.00761212, 0.00647648, 0.00664124, 0.01007852, 0.01141255,
       0.01120778, 0.01142937, 0.01032199, 0.01097575, 0.00900409,
       0.00893844, 0.01097988, 0.0090042 , 0.00509437, 0.00505479,
       0.00954922, 0.00936469, 0.01068033, 0.01110557, 0.01074242,
       0.00595171, 0.00567804, 0.00883304, 0.00451183, 0.01931211,
       0.01940251, 0.01740924, 0.0114285 , 0.01842117, 0.0300241 ,
       0.01903939, 0.02013313, 0.02067396, 0.01450535, 0.01768

In [202]:
(np.exp(-1 * s_lag[mask]) * w_lag[yard][mask])[0].shape

(103,)

In [212]:
(np.exp(-1 * s_lag_tile) * w_lag_tile[0][yard]).shape

(104, 104, 104)

In [210]:
s_lag_tile.shape

(104, 104, 1)

In [208]:
w_lag[yard][mask].shape

(103,)

In [209]:
w_lag_tile[0][yard].shape

(104,)

In [142]:
w_lag_tile[yard].shape

(104, 104)

## end test

In [100]:
dispersal_component_fast = 1 * np.nansum(((a_lag_tile * (y_lag_tile / n_lag_tile)) * np.exp(-1 * s_lag_tile) * (w_lag_tile[0] * np.exp(-1 * dist_tile[0])) * sI1_lag_tile), axis=1)

In [34]:
np.exp(-1 * dist[1])

array([0.9215137 , 1.        , 0.89074552, 0.67604991, 0.90210116,
       0.65754088, 0.68393053, 0.01597094, 0.00741065, 0.01377428,
       0.00769215, 0.00960061, 0.00763043, 0.01169124, 0.79415948,
       0.72318582, 0.57350488, 0.00284836, 0.00242028, 0.00251572,
       0.7469656 , 0.64888744, 0.00251149, 0.00279056, 0.00612368,
       0.01491806, 0.01321727, 0.00570911, 0.00241926, 0.00238814,
       0.00208911, 0.00348472, 0.00301917, 0.00000007, 0.        ,
       0.00000009, 0.00000013, 0.00000009, 0.00000003, 0.0000001 ,
       0.00000007, 0.00000001, 0.00000001, 0.00000004, 0.00000006,
       0.00000005, 0.00000007, 0.00000005, 0.00000006, 0.00000002,
       0.00000002, 0.00000007, 0.00000003, 0.00004288, 0.00004088,
       0.0000438 , 0.00004529, 0.000051  , 0.00006211, 0.00007344,
       0.00008533, 0.00011187, 0.00003404, 0.00004628, 0.05375955,
       0.05060382, 0.05901108, 0.07096751, 0.1674698 , 0.30931073,
       0.33304394, 0.25384438, 0.29250838, 0.23035286, 0.08608

In [35]:
np.exp(-1 * dist_tile[1])[:,1]

array([0.9215137 ,        nan, 0.89074552, 0.67604991, 0.90210116,
       0.65754088, 0.68393053, 0.01597094, 0.00741065, 0.01377428,
       0.00769215, 0.00960061, 0.00763043, 0.01169124, 0.79415948,
       0.72318582, 0.57350488, 0.00284836, 0.00242028, 0.00251572,
       0.7469656 , 0.64888744, 0.00251149, 0.00279056, 0.00612368,
       0.01491806, 0.01321727, 0.00570911, 0.00241926, 0.00238814,
       0.00208911, 0.00348472, 0.00301917, 0.00000007, 0.        ,
       0.00000009, 0.00000013, 0.00000009, 0.00000003, 0.0000001 ,
       0.00000007, 0.00000001, 0.00000001, 0.00000004, 0.00000006,
       0.00000005, 0.00000007, 0.00000005, 0.00000006, 0.00000002,
       0.00000002, 0.00000007, 0.00000003, 0.00004288, 0.00004088,
       0.0000438 , 0.00004529, 0.000051  , 0.00006211, 0.00007344,
       0.00008533, 0.00011187, 0.00003404, 0.00004628, 0.05375955,
       0.05060382, 0.05901108, 0.07096751, 0.1674698 , 0.30931073,
       0.33304394, 0.25384438, 0.29250838, 0.23035286, 0.08608

In [98]:
mask = np.arange(N) != 0 # mask out the current yard i
                
dispersal_component1 = 1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-1 * s_lag[mask]) * (w_lag[0][mask] * np.exp(-1 * dist[0][mask])) * sI1_lag[mask]))

In [99]:
dispersal_component1

0.3737640150555859

# code begins here

In [ ]:
import numpy as np
#from autograd import grad
import matplotlib.pyplot as plt

#import tensorflow as tf
#import torch
np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

theta_list = []
checkpoint_list = []

year = [2014]
period = ['may-jun'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')

    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')

    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')

    N = X['N']

    dist = X['distance']
    tI1 = X['tI1'].reshape(N,1)
    tI2 = X['tI2'].reshape(N,1)
    sI2 = X['sI2'].reshape(N,1)
    
    y_apr = X['y_apr'].reshape(N,1)
    y_may = X['y_may'].reshape(N,1)
    y_jun = X['y_jun'].reshape(N,1)
    y_jul = X['y_jul'].reshape(N,1)

    n_apr = X['n_apr'].reshape(N,1)
    n_may = X['n_may'].reshape(N,1)
    n_jun = X['n_jun'].reshape(N,1)
    n_jul = X['n_jul'].reshape(N,1)

    a_apr = X['a_apr'].reshape(N,1)
    a_may = X['a_may'].reshape(N,1)
    a_jun = X['a_jun'].reshape(N,1)
    a_jul = X['a_jul'].reshape(N,1)

    w_apr = X['wind_apr']
    w_may = X['wind_may']
    w_jun = X['wind_jun']
    w_jul = X['wind_jul']

    sI1_apr = X['sI1_apr'].reshape(N,1)
    sI1_may = X['sI1_may'].reshape(N,1)
    sI1_jun = X['sI1_jun'].reshape(N,1)
    sI1_jul = X['sI1_jul'].reshape(N,1)

    s_apr = X['s_apr'].reshape(N,1)
    s_may = X['s_may'].reshape(N,1)
    s_jun = X['s_jun'].reshape(N,1)
    s_jul = X['s_jul'].reshape(N,1)

    for period in period:

        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun
        
        a_lag_tile = np.tile(a_lag, (N, 1))
        y_lag_tile = np.tile(y_lag, (N, 1))
        n_lag_tile = np.tile(n_lag, (N, 1))
        s_lag_tile = np.tile(s_lag, (N, 1))
        w_lag_tile = np.tile(w_lag, (N, 1))
        dist_tile = np.tile(dist, (N, 1))
        sI1_lag_tile = np.tile(sI1_lag, (N, 1))
        sI2_tile = np.tile(sI2, (N, 1))
        
        # Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
        def eta(theta):
            
            eta = []
                
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            for i in range(0, N):
                
                mild_inc_lag = y_lag[i] / n_lag[i]
                s_lag_i = s_lag[i]
                
                auto_infection1 = beta1 + delta1 * (mild_inc_lag) * np.exp(-eta11 * s_lag_i)
                auto_infection2 = beta2 + delta2 * (mild_inc_lag) * np.exp(-eta12 * s_lag_i)

                mask = np.arange(N) != i # mask out the current yard i
                
                mild_inc_lag_mask = y_lag[mask] / n_lag[mask]
                a_lag_mask = a_lag[mask]
                a_times_mild_inc_lag_mask = a_lag_mask * mild_inc_lag_mask
                s_lag_mask = s_lag[mask]
                w_lag_i_mask = w_lag[i][mask]
                dist_i_mask = dist[i][mask]
                
                dispersal_component1 = gamma1 * np.sum((a_times_mild_inc_lag_mask * np.exp(-eta21 * s_lag_mask) * (w_lag_i_mask * np.exp(-alpha1 * dist_i_mask)) * sI1_lag[mask]))
                dispersal_component2 = gamma2 * np.sum((a_times_mild_inc_lag_mask * np.exp(-eta22 * s_lag_mask) * (w_lag_i_mask * np.exp(-alpha2 * dist_i_mask)) * sI2[mask]))
            
                eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
                eta.append(eta_i)
            
            eta = np.array(eta).reshape(N,1)
            
            return eta


        def costFunction(theta): 
            
            neg_log_likelihood = -(1/N) * np.sum(y * eta(theta) - n * np.log(1 + np.exp(eta(theta))))

            return neg_log_likelihood

from scipy.optimize import minimize

# Define initial parameter values
init_params = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

# Minimize the cost function using the L-BFGS-B method
result = minimize(costFunction, init_params, method='L-BFGS-B')

# Get the optimized parameter values
theta_opt = result.x

# Print the optimized parameter values
print("Optimized parameter values: ", theta_opt)

def partial(theta):
    
    beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
    
    d_beta1 = tI1
    d_beta2 = tI2
    
    d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
    d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
    
    d_gamma1 = []
    d_gamma2 = []
    d_alpha1 = []
    d_alpha2 = []
    d_eta21 = []
    d_eta22 = []
    
    for i in range(0, N):
        
        mask = np.arange(N) != i # mask out the current yard i
    
        d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask])
        d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask])
        
        d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
        d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])
        
        d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
        d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
    
        d_gamma1.append(d_gamma1_i)
        d_gamma2.append(d_gamma2_i)
        d_alpha1.append(d_alpha1_i)
        d_alpha2.append(d_alpha2_i)
        d_eta21.append(d_eta21_i)
        d_eta22.append(d_eta22_i)
    
    d_gamma1 = np.array(d_gamma1).reshape(N,1)
    d_gamma2 = np.array(d_gamma2).reshape(N,1)
    d_alpha1 = np.array(d_alpha1).reshape(N,1)
    d_alpha2 = np.array(d_alpha2).reshape(N,1)
    d_eta21 = np.array(d_eta21).reshape(N,1)
    d_eta22 = np.array(d_eta22).reshape(N,1)
    
    
    d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
    d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)



    grad_entries = np.array([d_beta1, d_beta2, d_delta1, d_delta2, d_gamma1, d_gamma2, d_alpha1, d_alpha2, d_eta11, d_eta12, d_eta21, d_eta22])
    
    return grad_entries

def partial_by_partial(theta):
    
    beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
    
    d_beta1 = tI1
    d_beta2 = tI2
    
    d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
    d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
    
    d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
    d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
    
    d_gamma1 = []
    d_gamma2 = []
    d_alpha1 = []
    d_alpha2 = []
    d_eta21 = []
    d_eta22 = []
    
    for i in range(0, N):
        
        mask = np.arange(N) != i # mask out the current yard i
    
        d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask])
        d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask])
        
        d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
        d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])

        d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
        d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
        
        d_gamma1.append(d_gamma1_i)
        d_gamma2.append(d_gamma2_i)
        d_alpha1.append(d_alpha1_i)
        d_alpha2.append(d_alpha2_i)
        d_eta21.append(d_eta21_i)
        d_eta22.append(d_eta22_i)
    
    d_gamma1 = np.array(d_gamma1).reshape(N,1)
    d_gamma2 = np.array(d_gamma2).reshape(N,1)
    d_alpha1 = np.array(d_alpha1).reshape(N,1)
    d_alpha2 = np.array(d_alpha2).reshape(N,1)
    d_eta21 = np.array(d_eta21).reshape(N,1)
    d_eta22 = np.array(d_eta22).reshape(N,1)
        

    grad_entries = np.array([[d_beta1*d_beta1, d_beta2*d_beta1, d_delta1*d_beta1, d_delta2*d_beta1, d_gamma1*d_beta1, d_gamma2*d_beta1, d_alpha1*d_beta1, d_alpha2*d_beta1, d_eta11*d_beta1, d_eta12*d_beta1, d_eta21*d_beta1, d_eta22*d_beta1],
                            [d_beta1*d_beta2, d_beta2*d_beta2, d_delta1*d_beta2, d_delta2*d_beta2, d_gamma1*d_beta2, d_gamma2*d_beta2, d_alpha1*d_beta2, d_alpha2*d_beta2, d_eta11*d_beta2, d_eta12*d_beta2, d_eta21*d_beta2, d_eta22*d_beta2],
                            [d_beta1*d_delta1, d_beta2*d_delta1, d_delta1*d_delta1, d_delta2*d_delta1, d_gamma1*d_delta1, d_gamma2*d_delta1, d_alpha1*d_delta1, d_alpha2*d_delta1, d_eta11*d_delta1, d_eta12*d_delta1, d_eta21*d_delta1, d_eta22*d_delta1],
                            [d_beta1*d_delta2, d_beta2*d_delta2, d_delta1*d_delta2, d_delta2*d_delta2, d_gamma1*d_delta2, d_gamma2*d_delta2, d_alpha1*d_delta2, d_alpha2*d_delta2, d_eta11*d_delta2, d_eta12*d_delta2, d_eta21*d_delta2, d_eta22*d_delta2],
                            [d_beta1*d_gamma1, d_beta2*d_gamma1, d_delta1*d_gamma1, d_delta2*d_gamma1, d_gamma1*d_gamma1, d_gamma2*d_gamma1, d_alpha1*d_gamma1, d_alpha2*d_gamma1, d_eta11*d_gamma1, d_eta12*d_gamma1, d_eta21*d_gamma1, d_eta22*d_gamma1],
                            [d_beta1*d_gamma2, d_beta2*d_gamma2, d_delta1*d_gamma2, d_delta2*d_gamma2, d_gamma1*d_gamma2, d_gamma2*d_gamma2, d_alpha1*d_gamma2, d_alpha2*d_gamma2, d_eta11*d_gamma2, d_eta12*d_gamma2, d_eta21*d_gamma2, d_eta22*d_gamma2],
                            [d_beta1*d_alpha1, d_beta2*d_alpha1, d_delta1*d_alpha1, d_delta2*d_alpha1, d_gamma1*d_alpha1, d_gamma2*d_alpha1, d_alpha1*d_alpha1, d_alpha2*d_alpha1, d_eta11*d_alpha1, d_eta12*d_alpha1, d_eta21*d_alpha1, d_eta22*d_alpha1],
                            [d_beta1*d_alpha2, d_beta2*d_alpha2, d_delta1*d_alpha2, d_delta2*d_alpha2, d_gamma1*d_alpha2, d_gamma2*d_alpha2, d_alpha1*d_alpha2, d_alpha2*d_alpha2, d_eta11*d_alpha2, d_eta12*d_alpha2, d_eta21*d_alpha2, d_eta22*d_alpha2],
                            [d_beta1*d_eta11, d_beta2*d_eta11, d_delta1*d_eta11, d_delta2*d_eta11, d_gamma1*d_eta11, d_gamma2*d_eta11, d_alpha1*d_eta11, d_alpha2*d_eta11, d_eta11*d_eta11, d_eta12*d_eta11, d_eta21*d_eta11, d_eta22*d_eta11],
                            [d_beta1*d_eta12, d_beta2*d_eta12, d_delta1*d_eta12, d_delta2*d_eta12, d_gamma1*d_eta12, d_gamma2*d_eta12, d_alpha1*d_eta12, d_alpha2*d_eta12, d_eta11*d_eta12, d_eta12*d_eta12, d_eta21*d_eta12, d_eta22*d_eta12],
                            [d_beta1*d_eta21, d_beta2*d_eta21, d_delta1*d_eta21, d_delta2*d_eta21, d_gamma1*d_eta21, d_gamma2*d_eta21, d_alpha1*d_eta21, d_alpha2*d_eta21, d_eta11*d_eta21, d_eta12*d_eta21, d_eta21*d_eta21, d_eta22*d_eta21],
                            [d_beta1*d_eta22, d_beta2*d_eta22, d_delta1*d_eta22, d_delta2*d_eta22, d_gamma1*d_eta22, d_gamma2*d_eta22, d_alpha1*d_eta22, d_alpha2*d_eta22, d_eta11*d_eta22, d_eta12*d_eta22, d_eta21*d_eta22, d_eta22*d_eta22]])
    
    
    
    return grad_entries

def partial_sq(theta):
    
    beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
    
    # delta1 second derivatives
    
    d_delta1_d_eta11 = -tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag) * s_lag
    d_delta1_d_eta12 = 0
    d_delta2_d_eta11 = 0
    d_delta2_d_eta12 = -tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag) * s_lag
    d_gamma1_d_eta22 = 0
    d_gamma1_d_alpha2 = 0
    d_gamma2_d_eta21 = 0
    d_gamma2_d_alpha1 = 0
    d_alpha1_d_gamma2 = 0
    d_alpha1_d_eta22 = 0
    d_alpha1_d_alpha2 = 0
    d_alpha2_d_gamma1 = 0
    d_alpha2_d_eta21 = 0
    d_alpha2_d_alpha1 = 0
    d_eta11_d_delta1 = -tI1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
    d_eta11_d_delta2 = 0
    d_eta11_d_eta11 = tI1 * delta1 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
    d_eta11_d_eta12 = 0
    d_eta12_d_delta1 = 0
    d_eta12_d_delta2 = -tI2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
    d_eta12_d_eta11 = 0
    d_eta12_d_eta12 = tI2 * delta2 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
    d_eta21_d_gamma2 = 0
    d_eta21_d_eta22 = 0
    d_eta21_d_alpha2 = 0
    d_eta22_d_gamma1 = 0
    d_eta22_d_eta21 = 0
    d_eta22_d_alpha1 = 0
    
    # summations
    
    d_gamma1_d_eta21 = []
    d_gamma1_d_alpha1 = []
    d_gamma2_d_eta22 = []
    d_gamma2_d_alpha2 = []
    d_alpha1_d_gamma1 = []
    d_alpha1_d_eta21 = []
    d_alpha1_d_alpha1 = []
    d_alpha2_d_gamma2 = []
    d_alpha2_d_eta22 = []
    d_alpha2_d_alpha2 = []
    d_eta21_d_gamma1 = []
    d_eta21_d_eta21 = []
    d_eta21_d_alpha1 = []
    d_eta22_d_gamma2 = []
    d_eta22_d_eta22 = []
    d_eta22_d_alpha2 = []
    
    
    for i in range(0, N):
        
        mask = np.arange(N) != i # mask out the current yard i
        
        d_gamma1_d_eta21_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
        d_gamma1_d_alpha1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
        d_gamma2_d_eta22_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
        d_gamma2_d_alpha2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])
        d_alpha1_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
        d_alpha1_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask] * s_lag[mask])
        d_alpha1_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * (dist[i][mask])**2)
        d_alpha2_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])
        d_alpha2_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask] * s_lag[mask])
        d_alpha2_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * (dist[i][mask])**2)
        d_eta21_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
        d_eta21_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * (s_lag[mask]**2))
        d_eta21_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask] * (dist[i][mask]))
        d_eta22_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
        d_eta22_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * (s_lag[mask]**2))
        d_eta22_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask] * (dist[i][mask]))

        d_gamma1_d_eta21.append(d_gamma1_d_eta21_i)
        d_gamma1_d_alpha1.append(d_gamma1_d_alpha1_i)
        d_gamma2_d_eta22.append(d_gamma2_d_eta22_i)
        d_gamma2_d_alpha2.append(d_gamma2_d_alpha2_i)
        d_alpha1_d_gamma1.append(d_alpha1_d_gamma1_i)
        d_alpha1_d_eta21.append(d_alpha1_d_eta21_i)
        d_alpha1_d_alpha1.append(d_alpha1_d_alpha1_i)
        d_alpha2_d_gamma2.append(d_alpha2_d_gamma2_i)
        d_alpha2_d_eta22.append(d_alpha2_d_eta22_i)
        d_alpha2_d_alpha2.append(d_alpha2_d_alpha2_i)
        d_eta21_d_gamma1.append(d_eta21_d_gamma1_i)
        d_eta21_d_eta21.append(d_eta21_d_eta21_i)
        d_eta21_d_alpha1.append(d_eta21_d_alpha1_i)
        d_eta22_d_gamma2.append(d_eta22_d_gamma2_i)
        d_eta22_d_eta22.append(d_eta22_d_eta22_i)
        d_eta22_d_alpha2.append(d_eta22_d_alpha2_i)
        
    d_gamma1_d_eta21 = np.array(d_gamma1_d_eta21).reshape((N, 1))
    d_gamma1_d_alpha1 = np.array(d_gamma1_d_alpha1).reshape((N, 1))
    d_gamma2_d_eta22 = np.array(d_gamma2_d_eta22).reshape((N, 1))
    d_gamma2_d_alpha2 = np.array(d_gamma2_d_alpha2).reshape((N, 1))
    d_alpha1_d_gamma1 = np.array(d_alpha1_d_gamma1).reshape((N, 1))
    d_alpha1_d_eta21 = np.array(d_alpha1_d_eta21).reshape((N, 1))
    d_alpha1_d_alpha1 = np.array(d_alpha1_d_alpha1).reshape((N, 1))
    d_alpha2_d_gamma2 = np.array(d_alpha2_d_gamma2).reshape((N, 1))
    d_alpha2_d_eta22 = np.array(d_alpha2_d_eta22).reshape((N, 1))
    d_alpha2_d_alpha2 = np.array(d_alpha2_d_alpha2).reshape((N, 1))
    d_eta21_d_gamma1 = np.array(d_eta21_d_gamma1).reshape((N, 1))
    d_eta21_d_eta21 = np.array(d_eta21_d_eta21).reshape((N, 1))
    d_eta21_d_alpha1 = np.array(d_eta21_d_alpha1).reshape((N, 1))
    d_eta22_d_gamma2 = np.array(d_eta22_d_gamma2).reshape((N, 1))
    d_eta22_d_eta22 = np.array(d_eta22_d_eta22).reshape((N, 1))
    d_eta22_d_alpha2 = np.array(d_eta22_d_alpha2).reshape((N, 1))
        
        
    
    zero = np.zeros((N, 1))
    
    hess_entries = np.array([[zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta1
                            [zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta2
                            [zero, zero, zero, zero, zero, zero, zero, zero, d_eta11_d_delta1, zero, zero, zero],    #d_delta1
                            [zero, zero, zero, zero, zero, zero, zero, zero, zero, d_eta12_d_delta2, zero, zero],    #d_delta2
                            [zero, zero, zero, zero, zero, zero, d_alpha1_d_gamma1, zero, zero, zero, d_eta21_d_gamma1, zero],    #d_gamma1
                            [zero, zero, zero, zero, zero, zero, zero, d_alpha2_d_gamma2, zero, zero, zero, d_eta22_d_gamma2],    #d_gamma2
                            [zero, zero, zero, zero, d_gamma1_d_alpha1, zero, d_alpha1_d_alpha1, zero, zero, zero, d_eta21_d_alpha1, zero],    #d_alpha1
                            [zero, zero, zero, zero, zero, d_gamma2_d_alpha2, zero, d_alpha2_d_alpha2, zero, zero, zero, d_eta22_d_alpha2],    #d_alpha2
                            [zero, zero, d_delta1_d_eta11, zero, zero, zero, zero, zero, d_eta11_d_eta11, zero, zero, zero],    #d_eta11
                            [zero, zero, zero, d_delta2_d_eta12, zero, zero, zero, zero, zero, d_eta12_d_eta12, zero, zero],    #d_eta12
                            [zero, zero, zero, zero, d_gamma1_d_eta21, zero, d_alpha1_d_eta21, zero, zero, zero, d_eta21_d_eta21, zero],    #d_eta21
                            [zero, zero, zero, zero, zero, d_gamma2_d_eta22, zero, d_alpha2_d_eta22, zero, zero, zero, d_eta22_d_eta22]])   #d_eta22
    
    
    return hess_entries



# Gradient entries
def gradient(theta):
    
    mu = y - (n / (1 + np.exp(-eta(theta))))
    
    # Gradient 
    gradient = - (1 / N) * np.sum((partial(theta) * mu), axis=1)
    
    return gradient


# Hessian
def hessian(theta):
    
    mu = y - (n / (1 + np.exp(-eta(theta))))
    
    # Hessian entries
    hessian = - (1 / N) * np.sum((partial_sq(theta) * mu - n * (partial_by_partial(theta)) * (np.exp(-eta(theta)) / (1 + np.exp(-eta(theta)))**2)), axis=2)
    hessian = hessian.reshape((12, 12))
    
    return hessian


Optimized parameter values:  [-2.53731989 -2.01494715  0.66066998  1.72186307  0.05244    -0.45389627
 -0.12492921  1.49349407 -0.01841706 -0.68115482  0.17942371  0.28993627]


In [8]:
gradient(theta_opt)

array([[  0.25682763],
       [  1.53018413],
       [ -0.18260122],
       [ -0.82936252],
       [ 32.66278026],
       [  1.59818026],
       [-32.76397433],
       [  0.4876694 ],
       [  0.11696976],
       [  1.71029156],
       [ -1.86248221],
       [  0.73577798]])

In [23]:



# Define the function eta() and cost function as before

# Set initial values for theta
theta = [0.1] * 12

# Minimize the cost function and get the optimized parameter values
res = minimize(costFunction, theta0, method='BFGS', options={'xtol': 1e-10, 'disp': True})
theta = res.x


Optimization terminated successfully.
         Current function value: 27.351807
         Iterations: 154
         Function evaluations: 2305
         Gradient evaluations: 177
Optimized theta:  [  -2.79611934   -2.60698314 1350.02147862    7.91181441    4.02480648
   -6.11303      -0.05246659    0.58942851    3.49828795   -1.19666649
  127.25299647   63.40396065]
Gradient at the optimized theta:  [ 0.         -0.00000119 -0.00000048 -0.00000048  0.         -0.00000048
 -0.00000334 -0.00000119  0.00000048  0.          0.          0.        ]


In [22]:
res.hes

AttributeError: hes

In [20]:
np.linalg.eigh(hessian(theta_opt), UPLO='L')[0]

array([  -0.        ,    0.        ,    0.        ,    0.00524136,
          0.01313857,    0.02160097,    0.54276867,    0.63385137,
          1.42988843,    5.21575017,   19.06613339, 1933.72214879])

In [19]:
costFunction(theta_opt)

27.35180693150504

In [18]:
gradient(theta_opt)

array([[ 0.00000027],
       [-0.00000105],
       [-0.00000014],
       [-0.0000001 ],
       [-0.00000006],
       [-0.00000003],
       [-0.00001749],
       [-0.0000012 ],
       [ 0.0000009 ],
       [ 0.00000039],
       [-0.        ],
       [-0.        ]])

In [1]:
import numpy as np
#from autograd import grad
import matplotlib.pyplot as plt

#import tensorflow as tf
#import torch
np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

theta_list = []
checkpoint_list = []

year = [2014, 2015, 2016, 2017]
period = ['may-jun', 'jun-jul'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')

    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')

    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')

    N = X['N']

    dist = X['distance']
    tI1 = X['tI1'].reshape(N,1)
    tI2 = X['tI2'].reshape(N,1)
    sI2 = X['sI2'].reshape(N,1)
    
    y_apr = X['y_apr'].reshape(N,1)
    y_may = X['y_may'].reshape(N,1)
    y_jun = X['y_jun'].reshape(N,1)
    y_jul = X['y_jul'].reshape(N,1)

    n_apr = X['n_apr'].reshape(N,1)
    n_may = X['n_may'].reshape(N,1)
    n_jun = X['n_jun'].reshape(N,1)
    n_jul = X['n_jul'].reshape(N,1)

    a_apr = X['a_apr'].reshape(N,1)
    a_may = X['a_may'].reshape(N,1)
    a_jun = X['a_jun'].reshape(N,1)
    a_jul = X['a_jul'].reshape(N,1)

    w_apr = X['wind_apr']
    w_may = X['wind_may']
    w_jun = X['wind_jun']
    w_jul = X['wind_jul']

    sI1_apr = X['sI1_apr'].reshape(N,1)
    sI1_may = X['sI1_may'].reshape(N,1)
    sI1_jun = X['sI1_jun'].reshape(N,1)
    sI1_jul = X['sI1_jul'].reshape(N,1)

    s_apr = X['s_apr'].reshape(N,1)
    s_may = X['s_may'].reshape(N,1)
    s_jun = X['s_jun'].reshape(N,1)
    s_jul = X['s_jul'].reshape(N,1)

    for period in period:

        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun
        

        # Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
        def eta(theta):
            
            eta = []
                
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            for i in range(0, N):
            
                auto_infection1 = beta1 + delta1 * (y_lag[i] / n_lag[i]) * np.exp(-eta11 * s_lag[i])
                auto_infection2 = beta2 + delta2 * (y_lag[i] / n_lag[i]) * np.exp(-eta12 * s_lag[i])

                mask = np.arange(N) != i # mask out the current yard i
                
                dispersal_component1 = gamma1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask]))
                dispersal_component2 = gamma2 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask]))
            
                eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
                eta.append(eta_i)
            
            eta = np.array(eta).reshape(N,1)
            
            return eta


        def costFunction(theta): 
            
            neg_log_likelihood = -(1/N) * np.sum(y * eta(theta) - n * np.log(1 + np.exp(eta(theta))))

            return neg_log_likelihood


        def partial(theta):
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            d_beta1 = tI1
            d_beta2 = tI2
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])
                
                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
            
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
            
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)



            grad_entries = np.array([d_beta1, d_beta2, d_delta1, d_delta2, d_gamma1, d_gamma2, d_alpha1, d_alpha2, d_eta11, d_eta12, d_eta21, d_eta22])
            
            return grad_entries

        def partial_by_partial(theta):
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            d_beta1 = tI1
            d_beta2 = tI2
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])

                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
                
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
                

            grad_entries = np.array([[d_beta1*d_beta1, d_beta2*d_beta1, d_delta1*d_beta1, d_delta2*d_beta1, d_gamma1*d_beta1, d_gamma2*d_beta1, d_alpha1*d_beta1, d_alpha2*d_beta1, d_eta11*d_beta1, d_eta12*d_beta1, d_eta21*d_beta1, d_eta22*d_beta1],
                                    [d_beta1*d_beta2, d_beta2*d_beta2, d_delta1*d_beta2, d_delta2*d_beta2, d_gamma1*d_beta2, d_gamma2*d_beta2, d_alpha1*d_beta2, d_alpha2*d_beta2, d_eta11*d_beta2, d_eta12*d_beta2, d_eta21*d_beta2, d_eta22*d_beta2],
                                    [d_beta1*d_delta1, d_beta2*d_delta1, d_delta1*d_delta1, d_delta2*d_delta1, d_gamma1*d_delta1, d_gamma2*d_delta1, d_alpha1*d_delta1, d_alpha2*d_delta1, d_eta11*d_delta1, d_eta12*d_delta1, d_eta21*d_delta1, d_eta22*d_delta1],
                                    [d_beta1*d_delta2, d_beta2*d_delta2, d_delta1*d_delta2, d_delta2*d_delta2, d_gamma1*d_delta2, d_gamma2*d_delta2, d_alpha1*d_delta2, d_alpha2*d_delta2, d_eta11*d_delta2, d_eta12*d_delta2, d_eta21*d_delta2, d_eta22*d_delta2],
                                    [d_beta1*d_gamma1, d_beta2*d_gamma1, d_delta1*d_gamma1, d_delta2*d_gamma1, d_gamma1*d_gamma1, d_gamma2*d_gamma1, d_alpha1*d_gamma1, d_alpha2*d_gamma1, d_eta11*d_gamma1, d_eta12*d_gamma1, d_eta21*d_gamma1, d_eta22*d_gamma1],
                                    [d_beta1*d_gamma2, d_beta2*d_gamma2, d_delta1*d_gamma2, d_delta2*d_gamma2, d_gamma1*d_gamma2, d_gamma2*d_gamma2, d_alpha1*d_gamma2, d_alpha2*d_gamma2, d_eta11*d_gamma2, d_eta12*d_gamma2, d_eta21*d_gamma2, d_eta22*d_gamma2],
                                    [d_beta1*d_alpha1, d_beta2*d_alpha1, d_delta1*d_alpha1, d_delta2*d_alpha1, d_gamma1*d_alpha1, d_gamma2*d_alpha1, d_alpha1*d_alpha1, d_alpha2*d_alpha1, d_eta11*d_alpha1, d_eta12*d_alpha1, d_eta21*d_alpha1, d_eta22*d_alpha1],
                                    [d_beta1*d_alpha2, d_beta2*d_alpha2, d_delta1*d_alpha2, d_delta2*d_alpha2, d_gamma1*d_alpha2, d_gamma2*d_alpha2, d_alpha1*d_alpha2, d_alpha2*d_alpha2, d_eta11*d_alpha2, d_eta12*d_alpha2, d_eta21*d_alpha2, d_eta22*d_alpha2],
                                    [d_beta1*d_eta11, d_beta2*d_eta11, d_delta1*d_eta11, d_delta2*d_eta11, d_gamma1*d_eta11, d_gamma2*d_eta11, d_alpha1*d_eta11, d_alpha2*d_eta11, d_eta11*d_eta11, d_eta12*d_eta11, d_eta21*d_eta11, d_eta22*d_eta11],
                                    [d_beta1*d_eta12, d_beta2*d_eta12, d_delta1*d_eta12, d_delta2*d_eta12, d_gamma1*d_eta12, d_gamma2*d_eta12, d_alpha1*d_eta12, d_alpha2*d_eta12, d_eta11*d_eta12, d_eta12*d_eta12, d_eta21*d_eta12, d_eta22*d_eta12],
                                    [d_beta1*d_eta21, d_beta2*d_eta21, d_delta1*d_eta21, d_delta2*d_eta21, d_gamma1*d_eta21, d_gamma2*d_eta21, d_alpha1*d_eta21, d_alpha2*d_eta21, d_eta11*d_eta21, d_eta12*d_eta21, d_eta21*d_eta21, d_eta22*d_eta21],
                                    [d_beta1*d_eta22, d_beta2*d_eta22, d_delta1*d_eta22, d_delta2*d_eta22, d_gamma1*d_eta22, d_gamma2*d_eta22, d_alpha1*d_eta22, d_alpha2*d_eta22, d_eta11*d_eta22, d_eta12*d_eta22, d_eta21*d_eta22, d_eta22*d_eta22]])
            
            
            
            return grad_entries

        def partial_sq(theta):
            
            beta1, beta2, delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            # delta1 second derivatives
            
            d_delta1_d_eta11 = -tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag) * s_lag
            d_delta1_d_eta12 = 0
            d_delta2_d_eta11 = 0
            d_delta2_d_eta12 = -tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag) * s_lag
            d_gamma1_d_eta22 = 0
            d_gamma1_d_alpha2 = 0
            d_gamma2_d_eta21 = 0
            d_gamma2_d_alpha1 = 0
            d_alpha1_d_gamma2 = 0
            d_alpha1_d_eta22 = 0
            d_alpha1_d_alpha2 = 0
            d_alpha2_d_gamma1 = 0
            d_alpha2_d_eta21 = 0
            d_alpha2_d_alpha1 = 0
            d_eta11_d_delta1 = -tI1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_delta2 = 0
            d_eta11_d_eta11 = tI1 * delta1 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_eta12 = 0
            d_eta12_d_delta1 = 0
            d_eta12_d_delta2 = -tI2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta12_d_eta11 = 0
            d_eta12_d_eta12 = tI2 * delta2 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta21_d_gamma2 = 0
            d_eta21_d_eta22 = 0
            d_eta21_d_alpha2 = 0
            d_eta22_d_gamma1 = 0
            d_eta22_d_eta21 = 0
            d_eta22_d_alpha1 = 0
            
            # summations
            
            d_gamma1_d_eta21 = []
            d_gamma1_d_alpha1 = []
            d_gamma2_d_eta22 = []
            d_gamma2_d_alpha2 = []
            d_alpha1_d_gamma1 = []
            d_alpha1_d_eta21 = []
            d_alpha1_d_alpha1 = []
            d_alpha2_d_gamma2 = []
            d_alpha2_d_eta22 = []
            d_alpha2_d_alpha2 = []
            d_eta21_d_gamma1 = []
            d_eta21_d_eta21 = []
            d_eta21_d_alpha1 = []
            d_eta22_d_gamma2 = []
            d_eta22_d_eta22 = []
            d_eta22_d_alpha2 = []
            
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
                
                d_gamma1_d_eta21_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
                d_gamma1_d_alpha1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
                d_gamma2_d_eta22_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
                d_gamma2_d_alpha2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])
                d_alpha1_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask])
                d_alpha1_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * dist[i][mask] * s_lag[mask])
                d_alpha1_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * (dist[i][mask])**2)
                d_alpha2_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask])
                d_alpha2_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * dist[i][mask] * s_lag[mask])
                d_alpha2_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * (dist[i][mask])**2)
                d_eta21_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask])
                d_eta21_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * (s_lag[mask]**2))
                d_eta21_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha1 * dist[i][mask])) * sI1_lag[mask] * s_lag[mask] * (dist[i][mask]))
                d_eta22_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask])
                d_eta22_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * (s_lag[mask]**2))
                d_eta22_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[i][mask] * np.exp(-alpha2 * dist[i][mask])) * sI2[mask] * s_lag[mask] * (dist[i][mask]))

                d_gamma1_d_eta21.append(d_gamma1_d_eta21_i)
                d_gamma1_d_alpha1.append(d_gamma1_d_alpha1_i)
                d_gamma2_d_eta22.append(d_gamma2_d_eta22_i)
                d_gamma2_d_alpha2.append(d_gamma2_d_alpha2_i)
                d_alpha1_d_gamma1.append(d_alpha1_d_gamma1_i)
                d_alpha1_d_eta21.append(d_alpha1_d_eta21_i)
                d_alpha1_d_alpha1.append(d_alpha1_d_alpha1_i)
                d_alpha2_d_gamma2.append(d_alpha2_d_gamma2_i)
                d_alpha2_d_eta22.append(d_alpha2_d_eta22_i)
                d_alpha2_d_alpha2.append(d_alpha2_d_alpha2_i)
                d_eta21_d_gamma1.append(d_eta21_d_gamma1_i)
                d_eta21_d_eta21.append(d_eta21_d_eta21_i)
                d_eta21_d_alpha1.append(d_eta21_d_alpha1_i)
                d_eta22_d_gamma2.append(d_eta22_d_gamma2_i)
                d_eta22_d_eta22.append(d_eta22_d_eta22_i)
                d_eta22_d_alpha2.append(d_eta22_d_alpha2_i)
                
            d_gamma1_d_eta21 = np.array(d_gamma1_d_eta21).reshape((N, 1))
            d_gamma1_d_alpha1 = np.array(d_gamma1_d_alpha1).reshape((N, 1))
            d_gamma2_d_eta22 = np.array(d_gamma2_d_eta22).reshape((N, 1))
            d_gamma2_d_alpha2 = np.array(d_gamma2_d_alpha2).reshape((N, 1))
            d_alpha1_d_gamma1 = np.array(d_alpha1_d_gamma1).reshape((N, 1))
            d_alpha1_d_eta21 = np.array(d_alpha1_d_eta21).reshape((N, 1))
            d_alpha1_d_alpha1 = np.array(d_alpha1_d_alpha1).reshape((N, 1))
            d_alpha2_d_gamma2 = np.array(d_alpha2_d_gamma2).reshape((N, 1))
            d_alpha2_d_eta22 = np.array(d_alpha2_d_eta22).reshape((N, 1))
            d_alpha2_d_alpha2 = np.array(d_alpha2_d_alpha2).reshape((N, 1))
            d_eta21_d_gamma1 = np.array(d_eta21_d_gamma1).reshape((N, 1))
            d_eta21_d_eta21 = np.array(d_eta21_d_eta21).reshape((N, 1))
            d_eta21_d_alpha1 = np.array(d_eta21_d_alpha1).reshape((N, 1))
            d_eta22_d_gamma2 = np.array(d_eta22_d_gamma2).reshape((N, 1))
            d_eta22_d_eta22 = np.array(d_eta22_d_eta22).reshape((N, 1))
            d_eta22_d_alpha2 = np.array(d_eta22_d_alpha2).reshape((N, 1))
                
                
            
            zero = np.zeros((N, 1))
            
            hess_entries = np.array([[zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta1
                                    [zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta2
                                    [zero, zero, zero, zero, zero, zero, zero, zero, d_eta11_d_delta1, zero, zero, zero],    #d_delta1
                                    [zero, zero, zero, zero, zero, zero, zero, zero, zero, d_eta12_d_delta2, zero, zero],    #d_delta2
                                    [zero, zero, zero, zero, zero, zero, d_alpha1_d_gamma1, zero, zero, zero, d_eta21_d_gamma1, zero],    #d_gamma1
                                    [zero, zero, zero, zero, zero, zero, zero, d_alpha2_d_gamma2, zero, zero, zero, d_eta22_d_gamma2],    #d_gamma2
                                    [zero, zero, zero, zero, d_gamma1_d_alpha1, zero, d_alpha1_d_alpha1, zero, zero, zero, d_eta21_d_alpha1, zero],    #d_alpha1
                                    [zero, zero, zero, zero, zero, d_gamma2_d_alpha2, zero, d_alpha2_d_alpha2, zero, zero, zero, d_eta22_d_alpha2],    #d_alpha2
                                    [zero, zero, d_delta1_d_eta11, zero, zero, zero, zero, zero, d_eta11_d_eta11, zero, zero, zero],    #d_eta11
                                    [zero, zero, zero, d_delta2_d_eta12, zero, zero, zero, zero, zero, d_eta12_d_eta12, zero, zero],    #d_eta12
                                    [zero, zero, zero, zero, d_gamma1_d_eta21, zero, d_alpha1_d_eta21, zero, zero, zero, d_eta21_d_eta21, zero],    #d_eta21
                                    [zero, zero, zero, zero, zero, d_gamma2_d_eta22, zero, d_alpha2_d_eta22, zero, zero, zero, d_eta22_d_eta22]])   #d_eta22
            
            
            return hess_entries



        # Gradient entries
        def gradient(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Gradient 
            gradient = - (1 / N) * np.sum((partial(theta) * mu), axis=1)
            
            return gradient


        # Hessian
        def hessian(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Hessian entries
            hessian = - (1 / N) * np.sum((partial_sq(theta) * mu - n * (partial_by_partial(theta)) * (np.exp(-eta(theta)) / (1 + np.exp(-eta(theta)))**2)), axis=2)
            hessian = hessian.reshape((12, 12))
            
            return hessian
        
        
        # Adam optimizer
        def adam(theta, alpha, num_iters, tolerance, b_1=0.9, b_2=0.999, eps=1e-8):

            # Make a copy of theta, to avoid changing the original array, since numpy arrays are passed by reference to functions
            theta = theta.copy()
            
            # Use a python list to save cost in every iteration
            J_history = []
            
            m = np.zeros(theta.shape)
            v = np.zeros(theta.shape)
            
            for i in range(num_iters):
                
                # Gradient
                g = gradient(theta)

                # First moment
                m = b_1 * m + (1 - b_1) * g

                # Second moment
                v = b_2 * v + (1 - b_2) * g**2

                # Bias correction for the first and second moments

                mhat = m / (1 - b_1**(i+1))

                vhat = v / (1 - b_2**(i+1))
                
                change = alpha * mhat / (np.sqrt(vhat) + eps)
                
                # Update parameter theta
                theta = theta - change
                
                # save the cost J in every iteration
                J_history.append(costFunction(theta))
                # report progress
                
                print('Year: ', year, 'Period: ', period, '#', i , 'cost: ', costFunction(theta), 'MaxGrad: ', np.max(np.abs(gradient(theta))), end='\r')
                #print('#', i , 'cost: ', costFunction(theta), 'MaxGrad: ', np.max(np.abs(gradient(theta))), 'MinEig: ', np.min(np.linalg.eigh(hessian(theta), UPLO='L')[0]), end='\r')
                
                if (np.max(np.abs(gradient(theta))) <= tolerance) & (np.min(np.linalg.eigh(hessian(theta), UPLO='L')[0]) > 0):
                   break
                
                #if (np.max(np.abs(gradient(theta))) <= tolerance):
                #    break
                
            return theta, J_history

            
        # Initialize fitting parameters

        # Xavier initialization for theta
        #theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))

        # Use below theta to continue from checkpoint
        #checkpoint = np.load('../reports/apr_may_2014.npy', allow_pickle=True)
        
        if year == 2014:
        
            alpha = 0.05
            tolerance = 1e-5
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
        
        if year == 2015:
            
            alpha = 0.05
            tolerance = 1e-5
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
        
        elif year == 2016:
        
            alpha = 0.05
            tolerance = 1e-5
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
        
        elif year == 2017:
            
            alpha = 0.05
            tolerance = 1e-5
            theta = np.random.normal(0, 1 / np.sqrt(N), size=(12, 1))
            
        #theta = np.load('../reports/theta2.npy')

        # Gradient descent settings
        iterations = 500000

        theta, J_history = adam(theta, alpha, iterations, tolerance)

        #print('iteration start:\t{:.3f}'.format(np.int32(checkpoint[1])))
        #print('previous final cost:\t{:.3f}'.format(checkpoint[2]))
        #print('updated final cost:\t{:.3f}'.format(J_history[-1]))
        #print('theta: \n', theta)

        #plt.plot(list(range(1, len(J_history) + 1)), J_history)
        #plt.xlabel('iterations')
        #plt.ylabel('cost')
        
        
        #plt.show()

        # Save trained parameters

        #iterations += checkpoint[1]
        checkpoint = np.array([theta, iterations, J_history], dtype=object)
        
        theta_list.append(theta)
        checkpoint_list.append(checkpoint)

        if (year == 2014) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2014.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2014.npy', checkpoint)
        
        elif (year == 2014) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2014.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2014.npy', checkpoint)
            
        elif (year == 2015) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2015.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2015.npy', checkpoint)
            
        elif (year == 2015) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2015.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2015.npy', checkpoint)
            
        elif (year == 2016) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2016.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2016.npy', checkpoint)
            
        elif (year == 2016) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2016.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2016.npy', checkpoint)
            
        elif (year == 2017) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2017.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2017.npy', checkpoint)
            
        elif (year == 2017) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2017.npy', theta)
            np.save('../reports/parameters/checkpoints/mle_checkpoint_jun-jul_2017.npy', checkpoint)

KeyboardInterrupt: 

In [2]:
eta([1,1,1,1,1,1,1,1,1,1,1,1])

array([[1.7646738 ],
       [1.73930528],
       [1.65473584],
       [1.93139259],
       [1.76540799],
       [1.3986976 ],
       [1.95839705],
       [1.70651168],
       [1.56203277],
       [1.66852604],
       [1.3919614 ],
       [1.50186376],
       [1.50950726],
       [1.59721265],
       [1.51706027],
       [1.90305011],
       [1.70262588],
       [1.29258475],
       [1.2444953 ],
       [1.29722595],
       [1.80194574],
       [1.86557007],
       [0.        ],
       [1.41130468],
       [1.29627529],
       [1.30713256],
       [1.29474232],
       [1.44411438],
       [1.30292913],
       [1.09135457],
       [1.16295066],
       [1.41997235],
       [1.47081181],
       [1.08635915],
       [1.00160987],
       [1.23699059],
       [1.21950372],
       [1.25817482],
       [1.02375102],
       [1.14058573],
       [1.12598965],
       [1.09155294],
       [1.04883961],
       [1.52459476],
       [1.52865701],
       [1.6136544 ],
       [0.        ],
       [1.652

## Function

$$
\eta_{i}=\sum_{k=1}^{K} I_{k}^{(t)}(i)\left[\beta_{k}+\delta_{k}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}} \exp{\left(-\eta_{1k} s_{i}\right)}\right)+\gamma_{k} \sum_{j=1}^{M_{i}}\left(\frac{a_{j} z_{j}}{n_{z_{j}}} \exp{\left(-\eta_{2k} s_{j}\right)} w_{i j} \exp{\left(-\alpha_{k} d_{i j}\right)} I_{k}^{(s)}(j)\right)\right]
$$

## Derivatives

$$
\begin{align*}
\frac{\partial \eta_{i}}{\partial \beta_{k}} &= I_{k}^{(t)}(i) \\

\frac{\partial \eta_{i}}{\partial \delta_{k}} &= I_{k}^{(t)}(i)\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}}\right) \exp \left(-\eta_{1 k} s_{i}\right) \\

\frac{\partial \eta_{i}}{\partial \eta_{1 k}} &= -I_{k}^{(t)}(i) \delta_{k} s_{i}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}}\right) \exp \left(-\eta_{1 k} s_{i}\right) \\

\frac{\partial \eta_{i}}{\partial \eta_{2 k}} &= -\gamma_{k} I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z j}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j) s_{j}\right] \\

\frac{\partial \eta_{i}}{\partial \gamma_{k}} &= I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z_{j}}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j)\right] \\

\frac{\partial \eta_{i}}{\partial \alpha_{k}} &= -\gamma_{k} I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z_{j}}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j) d_{i j}\right]
\end{align*}
$$

### Cost Function

$$J(\theta) = 
-\frac{1}{N} \sum_{i=1}^{N} y_{i} \eta_{i}-n_{i} \log \left(1+e^{\eta_{i}}\right)
$$

### Gradient

$$\frac{\partial J}{\partial \theta} =
-\frac{1}{N}\sum_{i=1}^{N} \frac{\partial \eta_{i}}{\partial \theta}\left(y_{i}-\frac{n_{i}}{1+e^{-\eta_{i}}}\right)
$$


### Hessian

$$\frac{\partial^2 J}{\partial \theta^2} = -\frac{1}{N}
\sum_{i=1}^{N}\left[\frac{\partial^{2} \eta_{i}}{\partial \theta^{2}}\left(y_{i}-\frac{n_{i}}{1+e^{-\eta_{i}}}\right)- n_{i} \left(\frac{\partial \eta_{i}}{\partial \theta}\right)^{2}\frac{e^{-\eta_{i}}}{\left(1+e^{-\eta_{i}}\right)^2}\right]
$$

## Optimization

In [50]:
#checkpoint = np.array([theta, iterations, J_history], dtype=object)
#np.save('../reports/checkpoint_June_July2_backup.npy', checkpoint)

In [45]:
checkpoint = np.load('../reports/checkpoint_June_July2.npy', allow_pickle=True)
theta = checkpoint[0]

In [7]:
theta

array([[-1.52503676],
       [-3.72457905],
       [ 0.65958768],
       [ 9.21605194],
       [-0.25269531],
       [ 0.03692741],
       [ 1.02650034],
       [ 0.84416864],
       [-0.35952126],
       [ 0.26510524],
       [ 1.39258634],
       [ 1.36603416]])

## Parameter Estimation

## Function

$$
\eta_{i}=\sum_{k=1}^{K} I_{k}^{(t)}(i)\left[\beta_{k}+\delta_{k}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}} \exp{\left(-\eta_{1k} s_{i}\right)}\right)+\gamma_{k} \sum_{j=1}^{M_{i}}\left(\frac{a_{j} z_{j}}{n_{z_{j}}} \exp{\left(-\eta_{2k} s_{j}\right)} w_{i j} \exp{\left(-\alpha_{k} d_{i j}\right)} I_{k}^{(s)}(j)\right)\right]
$$

$$\beta_1 = -2.89902093$$
$$\beta_2 = -4.33376942$$
$$\delta_1 = 3.86406603$$
$$\delta_2 = 7.1830044$$
$$\gamma_1 = 0.06209235$$
$$\gamma_2 = 6.21221296$$
$$\alpha_1 = 0.17578305$$
$$\alpha_2 = 1.31264131$$
$$\eta_{11} = 0.13978209$$
$$\eta_{12} = 0.40521989$$
$$\eta_{21} = -0.79182359$$
$$\eta_{22} = 0.55742334$$

## Eigenvalues of Hessian

In [10]:
np.linalg.eigvalsh(hessian(theta))

array([-0.01465556, -0.00072991,  0.00561519,  0.02470412,  0.09781676,
        0.24491857,  0.25933526,  0.32607831,  4.13126881,  5.13835828,
       58.13333007, 84.10888156])

## Gradient

In [11]:
gradient(theta)

array([[ 0.00035165],
       [ 0.00000092],
       [ 0.00093506],
       [-0.00000046],
       [ 0.00370447],
       [-0.00027325],
       [-0.00982039],
       [-0.00226836],
       [-0.00229518],
       [ 0.00000655],
       [-0.00047533],
       [ 0.00037915]])

In [31]:
nGrad = nd.Gradient(costFunction)
nHess = nd.Hessian(costFunction)
hess_theta = nHess(theta.reshape(12,))
grad_theta = nGrad(theta.reshape(12,))

In [10]:
def prob(theta):
    
    p = 1 / (1 + np.exp(-eta(theta)))
    
    return p

In [12]:
print('estimated probability of disease: \n', prob(theta))

estimated probability of disease: 
 [[0.02611759]
 [0.02631787]
 [0.02620498]
 [0.03368977]
 [0.02623533]
 [0.02624147]
 [0.02745905]
 [0.02710209]
 [0.04514933]
 [0.04289552]
 [0.16258389]
 [0.03171659]
 [0.15754236]
 [0.03329544]
 [0.02628519]
 [0.02748307]
 [0.02906667]
 [0.02786479]
 [0.02657169]
 [0.02892042]
 [0.02622809]
 [0.04582204]
 [0.5       ]
 [0.84480418]
 [0.24196183]
 [0.2553849 ]
 [0.33620011]
 [0.02684969]
 [0.49306239]
 [0.80226981]
 [0.53574716]
 [0.41690551]
 [0.91858263]
 [0.14165495]
 [0.        ]
 [0.02439342]
 [0.02535986]
 [0.02457473]
 [0.02567178]
 [0.17610494]
 [0.02407955]
 [0.01639678]
 [0.01495841]
 [0.02234735]
 [0.02345555]
 [0.02276539]
 [0.5       ]
 [0.02272612]
 [0.02457618]
 [0.11173597]
 [0.10794024]
 [0.22234288]
 [0.14008519]
 [0.2470191 ]
 [0.24700966]
 [0.02734554]
 [0.25025533]
 [0.02731197]
 [0.2534015 ]
 [0.02732704]
 [0.24922973]
 [0.02735182]
 [0.02728799]
 [0.02733912]
 [0.02321656]
 [0.02306974]
 [0.02242573]
 [0.5       ]
 [0.02569351

In [13]:
prob(theta).shape

(104, 1)